In [ ]:
!python -version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Скачать новый архив
!rm fatigue-detection.zip
!ls -l ~/.kaggle
!mkdir -p ~/.kaggle
!cd "/content/drive/My Drive/kaggle"
!cp "/content/drive/My Drive/kaggle/kaggle.json" ~/.kaggle
!kaggle datasets download -d timmjy/fatigue-detection -p /content
# !rm "fatigue-detection.zip"
# !wget https://www.kaggle.com/timmjy/fatigue-detection/download --output-document fatigue-detection.zip

rm: cannot remove 'fatigue-detection.zip': No such file or directory
ls: cannot access '/root/.kaggle': No such file or directory
 98% 819M/836M [00:07<00:00, 48.6MB/s]
100% 836M/836M [00:07<00:00, 114MB/s] 


In [ ]:
dataset_directory = "/content/datasets/fatigue-detection"
#Разархивировать
!rm -rf "/content/datasets/fatigue-detection"
!mkdir /content/datasets
!unzip -u "/content/fatigue-detection.zip" -d "/content/datasets/fatigue-detection"


In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder
import dlib


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def normalize_faces(img_path, out_resolution):
    image = cv2.imread(img_path)
    tagged_image = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor= 1.1,
        minNeighbors= 2,
        minSize=(10, 10)
    )

    results = []
    faces_with_eyes = 0;
    for (x, y, w, h) in faces:
        eyes = eyes_cascade.detectMultiScale(gray[y:y+h, x:x+w])
        if len(eyes) < 2:
          continue
        faces_with_eyes += 1
        
        p = int(w / 10)  # Отступ от лица - 10%
        if y - p + 1 <= 0 or x - p + 1 <= 0:
          p = 0
        img_cropped = image[y - p + 1:y + h + p, x - p + 1:x + w + p]
        img_rezised = cv2.resize(img_cropped, (out_resolution, out_resolution), interpolation=cv2.INTER_LINEAR)
        norm_img = np.zeros((out_resolution, out_resolution))
        norm_img = cv2.normalize(img_rezised, norm_img, 0, 255, cv2.NORM_MINMAX)
        results.append(norm_img)
    return (results, tagged_image)

def image_preparation(img_path, new_img_path):
  faces, tagged_photo = normalize_faces(img_path, 250)
  for ind, face in enumerate(faces):
    if len(faces) == 1:
      os.makedirs(os.path.dirname(new_img_path), exist_ok=True)
      cv2.imwrite(new_img_path, faces[0])
      return True
    elif len(faces) == 0:
      print(f"{bcolors.FAIL}Face not found " + img_path + f"{bcolors.ENDC}")
    else:
      print(f"Multiple face found " + img_path)
    return False


In [ ]:
# def setup(base, prepare_image_base):
#     paths = []
#     labels = []
#     states = ['alert', 'non_vigilant', 'tired']
#     for label in tqdm(states):
#         temp_base = base + f'/{label}'
#         prepare_image_temp_base = prepare_image_base + f'/{label}'
#         for img in tqdm(os.listdir(temp_base)):

#             if image_preparation(temp_base + f'/{img}', prepare_image_temp_base + f'/{img}') :
#               paths.append(prepare_image_temp_base + f'/{img}')
#               labels.append(label)

#     enc = OneHotEncoder(sparse = False)
#     labels = np.reshape(labels, (-1, 1))
#     enc.fit(labels)
#     labels = enc.transform(labels)
#     return np.array(paths), labels

# train_paths , train_labels = setup(dataset_directory + '/output/train', dataset_directory + '/prepare_output/train')
# test_paths , test_labels = setup(dataset_directory + '/output/test', dataset_directory + '/prepare_output/test')
# val_paths , val_labels = setup(dataset_directory + '/output/val', dataset_directory + '/prepare_output/val')

def setup(base):
    paths = []
    labels = []
    states = ['alert', 'non_vigilant', 'tired']
    for label in tqdm(states):
        temp_base = base + f'/{label}'
        for img in os.listdir(temp_base):
            paths.append(temp_base + f'/{img}')
            labels.append(label)
            
    enc = OneHotEncoder(sparse = False)
    labels = np.reshape(labels, (-1, 1))
    enc.fit(labels)
    print(enc.categories_)
    labels = enc.transform(labels)
    return np.array(paths), labels

train_paths , train_labels = setup(dataset_directory + '/output/train')
val_paths , val_labels = setup(dataset_directory + '/output/val')



[array(['alert', 'non_vigilant', 'tired'], dtype='<U12')]



[array(['alert', 'non_vigilant', 'tired'], dtype='<U12')]


In [ ]:
!cd "/content/datasets/fatigue-detection/prepare_output"
!zip -r "/content/datasets/fatigue-detection/prepare_output.zip" "/content/datasets/fatigue-detection/prepare_output"

In [ ]:
import keras,os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.utils.np_utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,GlobalAveragePooling2D,Concatenate, ReLU, LeakyReLU,Reshape, Lambda
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import metrics
from skimage.transform import resize

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
detector = dlib.get_frontal_face_detector()

class fatigue(tf.keras.utils.Sequence):
    def __init__(self, paths, labels, batch_size, shape, shuffle = True):
        self.paths, self.labels = paths, labels
        self.batch_size = batch_size
        self.shape = shape
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        #print(1, len(self))
        indexes = self.indexes[idx * self.batch_size : (idx+1) * self.batch_size]

        paths = self.paths[indexes]
        X = np.zeros((paths.shape[0], self.shape[0], self.shape[1], self.shape[2])) 
        # Generate data
        for i, path in enumerate(paths):
            X[i] = self.load_image(path)

        y = self.labels[indexes]
        return X,y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __iter__(self):
        """Create a generator that iterate over the Sequence."""
        for item in (self[i] for i in range(len(self))):
            yield item
            
    def load_image(self, path):
        image = cv2.imread(path)
        #plt.imshow(image)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        #print(len(faces))
        if len(faces) > 0:
            tl_x = faces[0].tl_corner().x
            tl_y = faces[0].tl_corner().y
            tl_h = faces[0].height()
            tl_w = faces[0].width()
            if tl_x > 0 and tl_y > 0 and tl_h > 10 and tl_w > 10:
                image = image[tl_y : tl_y + tl_w, tl_x:tl_x + tl_h, :]
        image_norm = image/255.0
        im = resize(image_norm, (self.shape[0], self.shape[1], self.shape[2]), mode='reflect')
        return im

In [ ]:
train_datagen = fatigue(train_paths, train_labels, batch_size=32, shape=(250, 250, 3))
val_datagen = fatigue(val_paths, val_labels, batch_size=32, shape=(250, 250, 3), shuffle=False)

In [ ]:
res_net = tf.keras.applications.ResNet152V2(
    include_top=False,
    weights="imagenet",
    input_shape=(250, 250, 3),
)

In [ ]:

model = Sequential()

res_net = tf.keras.applications.ResNet152V2(
    include_top=False,
    pooling = 'avg',
    weights="imagenet",
)
model.add(res_net)
model.add(Dense(3, activation = 'softmax'))

# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Functional)     (None, 2048)              58331648  
_________________________________________________________________
dense (Dense)                (None, 3)                 6147      
Total params: 58,337,795
Trainable params: 6,147
Non-trainable params: 58,331,648
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit_generator(
    train_datagen,
    steps_per_epoch=len(train_datagen),
    validation_data=val_datagen,
    #validation_steps=64,
    #class_weight = class_weights,
    epochs=20,
    #class_weight=class_weights,
    #workers=workers,
    verbose=2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
98/98 - 722s - loss: 0.9923 - acc: 0.4862 - val_loss: 0.8791 - val_acc: 0.6093
Epoch 2/20
98/98 - 695s - loss: 0.8087 - acc: 0.6581 - val_loss: 0.7943 - val_acc: 0.6812
Epoch 3/20
98/98 - 689s - loss: 0.7168 - acc: 0.7231 - val_loss: 0.7373 - val_acc: 0.7224
Epoch 4/20
98/98 - 700s - loss: 0.6490 - acc: 0.7727 - val_loss: 0.7312 - val_acc: 0.6941
Epoch 5/20
98/98 - 696s - loss: 0.5953 - acc: 0.7945 - val_loss: 0.6703 - val_acc: 0.7404
Epoch 6/20
98/98 - 691s - loss: 0.5517 - acc: 0.8198 - val_loss: 0.6496 - val_acc: 0.7455
Epoch 7/20
98/98 - 692s - loss: 0.5230 - acc: 0.8307 - val_loss: 0.6305 - val_acc: 0.7404
Epoch 8/20
98/98 - 689s - loss: 0.4968 - acc: 0.8339 - val_loss: 0.6353 - val_acc: 0.7609
Epoch 9/20
98/98 - 691s - loss: 0.4662 - acc: 0.8595 - val_loss: 0.5953 - val_acc: 0.7815
Epoch 10/20
98/98 - 693s - loss: 0.4367 - acc: 0.8748 - val_loss: 0.5784 - val_acc: 0.7841
Epoch 11/20
98/98 - 686s - loss: 0.4203 - acc: 0.8793 - val_loss: 0.5754 - val_acc: 0.7892
Epoch 12

In [ ]:
# plot the loss
plt.plot(model.history['loss'], label='train loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

NameError: ignored

In [ ]:
# plot the accuracy
plt.plot(model.history['precision'], label='test precision')
plt.legend()
plt.show()
plt.savefig('PrecisionVal')